In [24]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aabd4debf0c8... 100% ▕████████████████▏ 1.1 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling a85fe2a2e58e... 100% ▕████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [25]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv


In [26]:
HOST = "http://localhost:11434"
llm = OpenAI(base_url=f'{HOST}/v1', api_key='ollama')
HEADERS = {"Content-Type": "application/json"}

MODEL_DEEPSEEK = "deepseek-r1:1.5b"
MODEL_LLAMA = "llama3.2"
MODEL_GPT4O = "gpt-4o"
MODEL_GPT4O_MINI = "gpt-4o-mini"

In [27]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class JSWebsite:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--headless")  # Run in headless mode
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        # service = Service(PATH_TO_CHROME_DRIVER)
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        # input("Please complete the verification in the browser and press Enter to continue...")
        # time.sleep(5)  # Wait for the page to load completely
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input", "nav", "footer", "header"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [44]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = """You are an AI assistant designed to evaluate an applicant's resume against a given job posting to determine their fit for the role. Your response should:
1. Provide a structured analysis
   - List 3 reasons why the applicant is a good fit for the job.
   - List 3 reasons why the applicant may not be a good fit for the job.
2. Ignore irrelevant content
   - Exclude navigation-related or non-informative text from both the job posting and the resume.
3. Score the applicant's fit
   - Assign a fit score between 0 and 100, where:
     - 100 = Perfect fit
     - 0 = No match
   - The score should be based on skills, experience, and qualifications relative to the job description.
   - Consider years of experience as a factor but not as a strict requirement (e.g., someone slightly below the experience requirement may still be a strong candidate).
   - Be very strict about if applicants have the required skills or relevant experience.
   - A score of 40 or below indicates a poor fit.
   - A score of 60 or above indicates a acceptable fit.
   - A score of 80 or above indicates a strong fit.
4. Provide Resume Improvement Suggestions
   - Offer 3 specific and actionable suggestions to improve the applicant’s resume.
   - Focus on aligning their resume better with the job posting by:
     - Highlighting relevant skills or projects.
     - Adjusting wording to match industry terms.
     - Emphasizing transferable experience.
5. Format the response in Markdown
   - Ensure readability with headers, bullet points, and clear separation of sections.
The Example Response Format should be in Markdown, using the default template below:

## Applicant Fit Analysis

Score: 75/100

### ✅ Strengths (Good Fit)
- Bullet point 1
- Bullet point 2
- Bullet point 3

### ❌ Weaknesses (Possible Gaps)
- Bullet point 1
- Bullet point 2
- Bullet point 3

### ✍️ Resume Improvement Suggestions
1. Suggestion 1
2. Suggestion 2
3. Suggestion 3
"""

In [30]:
def user_prompt_for(resume, job_post_url):
    job_post = JSWebsite(job_post_url)
    user_prompt = f"Given the applicant profile and job post; \n"
    user_prompt += f"Job Post: {job_post.text}\n"
    user_prompt += f"Resume: {resume}\n"
    return user_prompt

In [31]:
# a python notebook vbox for uploading pdf files
from PyPDF2 import PdfReader

def read_pdf(path):
    text = ""
    with open(path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [32]:
yuan_resume = read_pdf('/Users/yualin/Desktop/Dropbox/資料/Resume/Resume Jan 30 2025.pdf')
# Let's try one out
genstudio_job_post_url = "https://adobe.wd5.myworkdayjobs.com/external_experienced/job/San-Jose/Software-Development-Engineer_R152283"
yuan_genstudio_prompt = user_prompt_for(yuan_resume, genstudio_job_post_url)

In [33]:
yuan_genstudio_prompt

"Given the applicant profile and job post; \nJob Post: \nResume: YU-AN LIN\nFull Stack Software Engineer\nKubernetes AdministratorUniversity of California, Santa Cruz\nMajor in Computer Science B.S.\nSep 2014 - Mar 2018\nGPA 3.81\nEXPERIENCEchockerlin+resume@gmail.com\nPage 1 / 2https://www.linkedin.com/in/yu-an-lin-19b94010b\nData Services - Dev Lead\n◦Led a team of five engineers to develop a data platform for an event-driven system on AEM author instances.\n◦Designed and implemented custom ETL pipelines to aggregate data from multiple platforms into a Snowflake data warehouse.\n◦Developed and integrated Sling filters to intercept HTTP events in AEM, enabling real-time user event generation.\nAEM Reporting Service - Distributed System\n◦Core maintainer of 5 Java Springboot microservices connected via Event-driven Kafka pipeline to support Adobe AEM statistics \ndata and reporting APIs\n◦Utilized Azure Data Explorer for storing user behavior events, CosmosDB for reporting schema, and 

In [34]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": yuan_genstudio_prompt}
]

In [35]:
llm = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [36]:
response = llm.chat.completions.create(
    model=MODEL_DEEPSEEK,
    messages=messages
)
display(Markdown(response.choices[0].message.content))

<think>
Alright, so I need to analyze why Yu-Yan Lin is a good fit for the full-stack software engineer role at Proofpoint. Let me start by understanding the job posting and looking through Yu-Yan's resume.

First, the position mentions a full-stack software engineer role at Proofpoint as being the leader developer of their serverless application using Java and Golang with Kubernetes. They have experience in leading teams, implementing security measures, and optimizing system performance. This suggests they're highly experienced in building scalable applications, especially on Kubernetes, which aligns well with the company's needs.

Looking at their resume, Yu-Yan has several relevant entries. They led a team of five engineers to develop a distributed data platform for AEM instances on AEM author instances, which directly ties into the full-stack role mentioned in the job posting. Their technical skills include ETL pipelining, Sling filters for event interception, and building microservices across multiple platforms. This shows they can manage large-scale systems effectively.

Also, their experience with React UI components using Adobe Micro-frontend suggests they have JavaScript proficiency, which is relevant for server-side applications often part of full-stack environments. Their hands-on approach in both data processing and container security makes them well-rounded in different programming languages like Java and the new Go language.

They mentioned being involved in developing distributed systems on Kubernetes, leveraging Azure Data Explorer, CosmosDB, and Blob Storage. This definitely shows expertise in cloud engineering and data storage solutions, which are key components of Kubernetes and Proofpoint's operations.

Additionally, they were involved in maintaining performance improvements by optimizing indexes, scaling data models with materialized views, and migrating from zip files to cube volumes. This indicates strong problem-solving skills and a focus on system optimization, both crucial for a full-stack role at Proofpoint.

Their experience with UI/UX tools like Vue.js, React, and Angular further supports their ability to manage development pipelines across different platforms, which is another important aspect of the job description.

Overall, Yu-Yan's extensive experience in large-scale systems, strong technical skills in relevant programming languages, effective team leadership in both data processing and security aspects, and a deep focus on system optimization aligns well with the full-stack software engineer role at Proofpoint. Their practical experience with Kubernetes, including leading distributed applications via AWS, also highlights their ability to lead complex teams in managing modern cloud systems.
</think>

Based on the detailed analysis of Yu-Yan Lin's resume and the job posting, here is a step-by-step summary of why/he is a good fit for the full-stack software engineer role at Proofpoint:

---

### **Step 1: Understanding the Role Requirements**
- **Focus Areas:** 
  - Building a distributed data platform.
  - Providing API documentation.
  - Optimizing performance metrics.
  - Managing security and system integrity.
- **Expectations:**
  - Ability to build large-scale distributed systems using Kubernetes.
  - Strong practical experience in developing serverless applications.
  - Proven ability to manage complex CI/CD pipelines across multiple platforms.

### **Step 2: Reviewing Yu-Yan Lin's Experience and Relevant Skills**
- **Years of Experience:**
  - Lead team: 17+ years (from Proofpoint's establishment until current position).
- **Key Responsibilities:**
  - Led team size of five to develop distributed data platform for AEM instances.
  - Developed APIs documentation ensuring security, integrity, and compliance.
  - Successfully transitioned internal API from zip files to cube volumes while optimizing performance metrics.
  - Maintained strong knowledge of both Java and Go (new language) across multiple platforms.
- **Technical Proficiency:**
  - ETL pipelining using Sling filters for event interception.
  - Built microservices on AWS clusters with Kubernetes, ensuring scale, resilience, and security.
  - Developed Golang controllers and operational services to manage a distributed system, leveraging Go for high-performance serverless computing.
    - Conducted container networking interface evaluations (CNI) solutions such as Cilumbo, Calico, and AWS CNI.
  - Built frontend components with React components using Vue.js, Angular, or Angular React.
    - Designed UI/UX tools for API development, user interfaces on iOS devices, and mobile optimization of web applications.

### **Step 3: Aligning Experience with the Role**
- The job posting emphasizes:
  - Building a data-driven agricultural IOT device (as a personal project).
  - Leading a client's serverless architecture using Golang and Kubernetes.
  - Managing distributed systems on AWS, specifically Proofpoint's cloud infrastructure.
- **Appropriate Areas of Focus:**
  - Building end-to-end distributed systems across different platforms and languages (Java to Golang) is highly relevant for the full-stack role at Proofpoint.
  - Problem-solving skills demonstrated through API development and container security are essential for managing secure and scalable applications.

### **Step 4: Highlighting the F focuses in Depth**
1. **Leadership and Team Management:**  
   - Lead a distributed system with up-to-date cloud infrastructure, demonstrating strong leadership qualities within Proofpoint's team of engineers.
2. **Kubernetes Mastery:**  
   - Worked with Kubernetes for container orchestration (Golang controllers), distributed systems, and microservices, aligning with the full-stack requirement of managing modern cloud environments at Proofpoint.
3. **Optimization and Scalability:**  
   - Developed high-performing microservices systems using Kubernetes and optimizing system performance to handle increasing demands.

### **Step 5: Final Thoughts**
- Yu-Yan Lin brings a proven ability to lead cross-platform distributed systems, optimize performance metrics, and manage security in cloud environments.
- Their technical proficiency with both Java and Go across multiple platforms aligns perfectly with the full-stack engineer role at Proofpoint.
- While the job posting emphasizes API development and system-level security, their experience with API documentation further reinforces this focus.

### **Conclusion:**
Yu-Yan Lin is highly qualified to serve as a full-stack software engineer for Proofpoint due to their extensive hands-on experience across distributed systems, cloud platforms (AWS), optimization techniques (Golang, Python), container orchestration (Kubernetes), and security practices. Their ability to effectively manage complex CI/CD pipelines, work with different programming languages, and collaborate in an organized team aligns perfectly with the requirements of Proofpoint's role.

In [37]:
response = llm.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)
display(Markdown(response.choices[0].message.content))

## Applicant Fit Analysis

Score: 85/100

### ✅ Strengths (Good Fit)
- **Expertise in Kubernetes Administration**: Yu-An Lin has significant experience in administering Kubernetes, including designing and implementing custom ETL pipelines and developing Sling filters for real-time user event generation.
- **Cloud Security Leadership**: With extensive leadership roles in security tasks encompassing container and Kubernetes security, Lin demonstrates a strong understanding of cloud security practices.
- **Full Stack Development Skills**: As a Full Stack Software Engineer, Yu-An Lin has expertise in various programming languages including Java, Node.js, Vue.js, and Golang.

### ❌ Weaknesses (Possible Gaps)
- **Limited Industry Experience**: Although Lin has worked in the Adobe industry for over two years, most of his experience comes from a single company (Adobe).
- **Variable Work Histories**: His work history includes several roles, which might be seen as a lack of commitment or stability by some employers.
- **Unclear Career Progression**: Limited details on career advancement and promotions within companies could indicate slower growth or unclear paths.

### 💡 Resume Improvement Suggestions
1. Consider adding more specific metrics or achievements in bullet points to make it easier for readers to visualize your contributions.
2. Emphasize transferable experience by explicitly stating how skills gained in one role can be applied to others.
3. Add quantifiable numbers to help quantify the impact of projects you worked on and results achieved.

#### Updated Resume Example:
**Data Services - Dev Lead**
- **Real-time User Event Generation**: Developed custom Sling filters for AEM, resulting in a 300% increase in real-time user data collection.
...

This will allow for better tracking of progress, accomplishments, and measurable impact. The suggestions listed can serve to improve clarity, metrics, and potential career progression.

```

In [38]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [39]:
openai = OpenAI()

In [40]:
response = openai.chat.completions.create(
    model=MODEL_GPT4O_MINI,
    messages=messages
)
display(Markdown(response.choices[0].message.content))

## Applicant Fit Analysis

Score: 85/100

### ✅ Strengths (Good Fit)
- **Strong Technical Experience**: The applicant has relevant experience working with Kubernetes, CI/CD, and container security, which aligns well with the job requirements. 
- **Leadership Skills**: Led teams and projects effectively, showcasing good leadership and project management abilities, specifically as a Dev Lead and during security initiatives.
- **Full Stack Development**: Experience in both frontend (React, Vanilla JS) and backend (Java Springboot, Node.js) development demonstrates versatility crucial for full-stack roles.

### ❌ Weaknesses (Possible Gaps)
- **Specific Tools and Technologies**: While the applicant has exposure to AWS, they might lack experience with other cloud platforms mentioned in job descriptions (if any), potentially limiting adaptability.
- **Years of Experience in Required Technologies**: Although the applicant has ample experience, they might not have enough years specifically focused on certain requested technologies compared to other candidates.
- **Lack of Mentioned Certifications**: The job posting may require or prefer certain industry certifications (e.g., AWS Certified Solutions Architect, specific Kubernetes certifications) which are not evident in the resume.

### ✍️ Resume Improvement Suggestions
1. **Highlight Certifications**: If the applicant has any relevant certifications, they should prominently feature them on the resume, especially related to Kubernetes, AWS, or any other pertinent technologies.
2. **Align Terminology with Job Posting**: Adjust wording to incorporate industry-specific jargon from the job posting to ensure compatibility and demonstrate alignment with the role’s requirements.
3. **Expand on Projects**: For personal projects and open-source contributions, provide more context on their relevance to the job description, especially focusing on the technologies and methodologies used that are directly applicable to the role.

In [41]:
# UI for upload file
# The job posting and the CV are required to define the user prompt
# The user will input the job posting as text in a box here
# The user will upload the CV in PDF format, from which the text will be extracted

# You might need to install PyPDF2 via pip if it's not already installed
# !pip install PyPDF2

import io
import time
import requests
import PyPDF2
from bs4 import BeautifulSoup
from ipywidgets import Text, FileUpload, Button, VBox, HTML

# Create widgets for job posting URL input
job_posting_url = Text(
    placeholder='Enter the job posting URL here...',
    description='Job URL:',
    disabled=False,
    layout={'width': '600px'}
)

# File upload for CV
cv_upload = FileUpload(
    accept='.pdf',  # Only accept PDF files
    multiple=False,  # Only allow single file selection
    description='Upload CV (PDF)'
)

# Status display
status = HTML(value="<b>Status:</b> Waiting for inputs...")

# Submit Button
submit_button = Button(description='Submit', button_style='success')

# Dictionary to store extracted data
for_user_prompt = {
    'job_posting': '',
    'cv_text': ''
}

# Function to process PDF and extract text
def extract_cv_text(uploaded_file):
    print("Extracting CV text...")
    try:
        content = io.BytesIO(uploaded_file['content'])
        print("Reading PDF content...")
        pdf_reader = PyPDF2.PdfReader(content)
        print("Extracting text from PDF...")
        cv_text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
        print("Text extraction complete.")
        return cv_text if cv_text else "xxxxError extracting text from CV."
    except Exception as e:
        return f"xxxxError processing PDF: {str(e)}"

# Function to handle submission
def submit_action(b):
    print("Submit button clicked.")
    status.value = "<b>Status:</b> Processing..."

    # Process job posting
    job_url = job_posting_url.value.strip()
    status.value = f"<b>Status:</b> URL is {job_url}..."

    for_user_prompt['job_posting'] = job_url

    # Process CV
    status.value = f"<b>Status:</b> Checking CV..."
    if cv_upload.value:
        status.value = f"<b>Status:</b> CV uploaded fine..."
        uploaded_file = cv_upload.value[0]
        status.value = f"<b>Status:</b> Extracting CV..."
        for_user_prompt['cv_text'] = extract_cv_text(uploaded_file)
        if "xxxxError" in for_user_prompt['cv_text']:
            status.value = f"<b>Status:</b> {for_user_prompt['cv_text']}"
            return
        status.value = f"<b>Status:</b> CV extracted successfully!"
    else:
        status.value = "<b>Status:</b> Please upload a CV before submitting."
        return

    # Final status update
    status.value = "<b>Status:</b> Job posting and CV submitted successfully!"

    # Print extracted data (for debugging)
    print("Extracted Job Posting Text:", for_user_prompt['job_posting'][:500])  # Show first 500 chars
    print("Extracted CV Text:", for_user_prompt['cv_text'][:500])


# Attach submit action to button
submit_button.on_click(submit_action)

# Layout
layout = VBox([
    HTML(value="<h3>Input Job Posting URL and Upload CV</h3>"),
    job_posting_url,
    cv_upload,
    submit_button,
    status
])

# Display the UI
display(layout)


In [46]:
# Send to LLM openai
# https://adobe.wd5.myworkdayjobs.com/external_experienced/job/San-Jose/Software-Development-Engineer_R152283
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_for(for_user_prompt['cv_text'], for_user_prompt['job_posting'])}
]
response = openai.chat.completions.create(
    model=MODEL_GPT4O_MINI,
    messages=messages
)
display(Markdown(response.choices[0].message.content))

## Applicant Fit Analysis

Score: 85/100

### ✅ Strengths (Good Fit)
- **Extensive Software Development Experience**: With over 15 years of experience in software engineering roles, the applicant has a strong background in developing and managing software applications, which aligns well with the requirements of many software engineering positions.
- **Technical Skills Alignment**: The applicant demonstrates strong proficiency in ReactJS, NodeJS, Java, and various other modern technologies and frameworks, matching the demands of a role requiring knowledge of contemporary software stacks.
- **Project Management and Problem-Solving Skills**: The experience of managing software projects and effectively troubleshooting complex software problems indicates solid capabilities in project management and problem resolution, essential for high-stakes engineering roles.

### ❌ Weaknesses (Possible Gaps)
- **Specific Technology Gaps**: While the applicant has a diverse skill set, any specific technologies or platforms mentioned in the job posting that are not listed in the resume could indicate a gap in required expertise.
- **Lack of Recent Trends**: Depending on the job's focus, there may be newer technologies or methodologies (e.g., microservices architecture) that are not prominently featured in the applicant's resume, which could be crucial for certain roles.
- **Limited Experience in Certain Areas**: If the role specifically emphasizes work in a collaborative environment or certain development methodologies (like Agile), the resume might lack explicit mentions of this experience.

### ✍️ Resume Improvement Suggestions
1. **Highlight Recent Technologies**: Ensure that any newer technologies, methodologies, and tools relevant to the job are emphasized in the resume. For example, if the job listing mentions microservices, the applicant should clarify any relevant experience.
2. **Include Collaboration and Agile Methodologies**: If the applicant has experience working in Agile teams or other collaborative environments, it should be explicitly mentioned to better align with roles that prioritize teamwork and Agile practices.
3. **Project Examples**: Adjust project descriptions to explicitly state the outcomes and impact of the projects. This can include metrics on performance improvements or user adoption to quantify success.

By highlighting relevant experience and aligning the language used with that of the field, the applicant can enhance their resume further and strengthen their candidacy for desired positions.